<a href="https://colab.research.google.com/github/alvachaitanya/CrowdMotionCNNModels/blob/master/MotionCNNClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries 

In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import random
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
cd drive/My\ Drive/

/content/drive/My Drive


# Data Pre Processing 

In [ ]:
path = "MII/miidata"
cat = ['Lanes', 'Bottlenecks', 'Blocks', 'Arcs']
catdict = {'Lanes' : [1,0,0,0], 'Bottlenecks': [0,1,0,0], 'Blocks': [0,0,1,0], 'Arcs': [0,0,0,1]}
imgpath = []
for i in cat:
  imglist = os.listdir(os.path.join(path,i))
  for j in imglist:
    imgpath.append(os.path.join(path,i,j))


In [ ]:
class miidataset(Dataset):
  def __init__(self, imgpath, train=True, transform=None, splitratio=0.3):
    self.train = train
    self.imgpath = imgpath
    self.splitratio = splitratio
    self.arr=[]
    self.transform = transform
    self.data_splitting()

  def __getitem__(self, idx):
    if self.train == True:
      img = np.array(io.imread(self.arr[0][idx]))
      for i in cat:
        if self.arr[0][idx].find(i) != -1:
          category = i
    else:
      img = np.array(io.imread(self.arr[1][idx]))
      for i in cat:
        if self.arr[1][idx].find(i) != -1:
          category = i
    
    category = np.array(catdict[category])
    
    if self.transform:
      img = img.transpose((2, 0, 1))
      img = torch.as_tensor(img/255)#, dtype = torch.float32)/255
      category = torch.as_tensor(category)
    
    return img, category
  
  def __len__(self):
    if self.train:
      return len(self.arr[0])
    else:
      return len(self.arr[1])

  def data_splitting(self):
    '''Splitting data into train and test dataset'''
    data = self.imgpath
    random.shuffle(data)
    no_test = int(self.splitratio * len(self.imgpath))
    test_data = data[:no_test]
    train_data = data[no_test:]
    self.arr = [train_data, test_data]


In [ ]:
x = miidataset(imgpath, train=True, transform=True)
print(x[10][0].shape, x[10][1])
print(len(x))

torch.Size([3, 224, 224]) tensor([0, 1, 0, 0])
167


# Data Loader

In [ ]:
# Define train and test set
trainset = miidataset(imgpath, transform = True, train= True, splitratio = 0.3)
train_loader = torch.utils.data.DataLoader(trainset, batch_size = 4, shuffle = True, num_workers = 2)

testset = miidataset(imgpath, transform = True, train= False, splitratio = 0.3)
test_loader = torch.utils.data.DataLoader(testset, batch_size = 4, shuffle = False, num_workers = 2)

# CNN Models

## VGG 11

In [ ]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11', pretrained = False)
print(model)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
print(model.classifier[6])
model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=4, bias=True)
print(model.classifier[6])

Linear(in_features=4096, out_features=1000, bias=True)
Linear(in_features=4096, out_features=4, bias=True)


In [ ]:
# Define optimzer and criterian
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()

In [ ]:
def train(train_loader, net, criterion, optimizer, epochs, args):
  # Setting network for training mode.
  net.train()
  
  # Average Meter for batch loss.
  train_loss = list()

  for epoch in range(epochs):
    # Iterating over batches.
    for i, data in enumerate(train_loader):
      # Obtaining images, labels and paths for batch.
      inps, labs = data
      
      # Casting tensors to cuda.
      inps, labs = inps.cuda(args['device']), labs.cuda(args['device'])

      inps.squeeze_(0) # create a mini-batch as expected by the model
      labs.squeeze_(0)
        
      # Casting to cuda variables.
      inps = Variable(inps).float().cuda(args['device'])
      labs = Variable(labs).float().cuda(args['device'])

      # Clears the gradients of optimizer.
      optimizer.zero_grad()

      # # Forwarding.
      outs = net(inps)
      soft_outs = F.softmax(outs, dim=1)
      
      # Obtaining predictions.
      prds = soft_outs.data.max(1)[1]
      
      # Computing loss.
      loss = criterion(outs, labs)
              
      # Computing backpropagation.
      loss.backward()
      optimizer.step()
          
      # Appending images for epoch loss calculation.
      
      prds = prds.cpu().numpy()        
      inps_np = inps.detach().cpu().numpy()
      labs_np = labs.detach().cpu().numpy()
      
    # Updating loss meter.
    train_loss.append(loss.data.item())
    # Printing.
    if (i + 1) % args['print_freq'] == 0:
        print('[epoch %d], [iter %d / %d], [train loss %.5f]' % (epoch, i + 1, len(train_loader), np.asarray(train_loss).mean()))
    torch.cuda.empty_cache()
  plt.plot(train_loss)
  plt.show()
  return net

In [ ]:
vgg_model_trained = train(train_loader, model.cuda('cuda'), criterion, optimizer, 50, {'device':'cuda', 'print_freq':3})


[epoch 0], [iter 42 / 42], [train loss 1.13188]
[epoch 1], [iter 42 / 42], [train loss 0.87142]
[epoch 2], [iter 42 / 42], [train loss 0.78903]
[epoch 3], [iter 42 / 42], [train loss 0.84515]
[epoch 4], [iter 42 / 42], [train loss 0.77908]
[epoch 5], [iter 42 / 42], [train loss 0.70210]
[epoch 6], [iter 42 / 42], [train loss 0.73438]
[epoch 7], [iter 42 / 42], [train loss 0.68364]
[epoch 8], [iter 42 / 42], [train loss 0.68565]
[epoch 9], [iter 42 / 42], [train loss 0.65437]
[epoch 10], [iter 42 / 42], [train loss 0.65984]
[epoch 11], [iter 42 / 42], [train loss 0.62060]
[epoch 12], [iter 42 / 42], [train loss 0.60159]
[epoch 13], [iter 42 / 42], [train loss 0.58224]
[epoch 14], [iter 42 / 42], [train loss 0.57730]
[epoch 15], [iter 42 / 42], [train loss 0.57188]
[epoch 16], [iter 42 / 42], [train loss 0.57507]
[epoch 17], [iter 42 / 42], [train loss 0.56527]
[epoch 18], [iter 42 / 42], [train loss 0.54704]
[epoch 19], [iter 42 / 42], [train loss 0.54689]
[epoch 20], [iter 42 / 42], [t

In [ ]:
plt.plot(train_loss)

NameError: ignored

In [ ]:
total =0
correct =0
for i, data in enumerate(test_loader):
  inps, labs = data
  inps, labs = inps.cuda('cuda'), labs.cuda('cuda')

  inps.squeeze_(0) # create a mini-batch as expected by the model
  labs.squeeze_(0)
    
  # Casting to cuda variables.
  inps = Variable(inps).float().cuda('cuda')
  labs = Variable(labs).float().cuda('cuda')
  net = vgg_model_trained.cuda('cuda')
  outs = net(inps)
  soft_outs = F.softmax(outs, dim=1)
  prds = soft_outs.data.max(1)[1]
  labs = labs.data.max(1)[1]
  for i in range(len(prds)):
    total += 1
    if prds[i] == labs[i]:
      correct += 1

acc = correct/total*100
print(f"The accuracy % is :{acc}")

The accuracy % is :97.1830985915493


## AlexNet

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)
model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
print(model.classifier[6])
model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=4, bias=True)
print(model.classifier[6])

Linear(in_features=4096, out_features=1000, bias=True)
Linear(in_features=4096, out_features=4, bias=True)


In [ ]:
# Define optimzer and criterian
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()

In [ ]:
def train(train_loader, net, criterion, optimizer, epochs, args):
  # Setting network for training mode.
  net.train()
  
  # Average Meter for batch loss.
  train_loss = list()

  for epoch in range(epochs):
    # Iterating over batches.
    for i, data in enumerate(train_loader):
      # Obtaining images, labels and paths for batch.
      inps, labs = data
      
      # Casting tensors to cuda.
      inps, labs = inps.cuda(args['device']), labs.cuda(args['device'])

      inps.squeeze_(0) # create a mini-batch as expected by the model
      labs.squeeze_(0)
        
      # Casting to cuda variables.
      inps = Variable(inps).float().cuda(args['device'])
      labs = Variable(labs).float().cuda(args['device'])

      # Clears the gradients of optimizer.
      optimizer.zero_grad()

      # # Forwarding.
      outs = net(inps)
      soft_outs = F.softmax(outs, dim=1)
      
      # Obtaining predictions.
      prds = soft_outs.data.max(1)[1]
      
      # Computing loss.
      loss = criterion(outs, labs)
              
      # Computing backpropagation.
      loss.backward()
      optimizer.step()
          
      # Appending images for epoch loss calculation.
      
      prds = prds.cpu().numpy()        
      inps_np = inps.detach().cpu().numpy()
      labs_np = labs.detach().cpu().numpy()
      
      # Updating loss meter.
      train_loss.append(loss.data.item())

      # Printing.
      if (i + 1) % args['print_freq'] == 0:
          print('[epoch %d], [iter %d / %d], [train loss %.5f]' % (epoch, i + 1, len(train_loader), np.asarray(train_loss).mean()))
      torch.cuda.empty_cache()
  return net

In [ ]:
alexNet_model_trained = train(train_loader, model.cuda('cuda'), criterion, optimizer, 1, {'device':'cuda', 'print_freq':3})

[epoch 0], [iter 3 / 42], [train loss 0.56385]
[epoch 0], [iter 6 / 42], [train loss 0.48459]
[epoch 0], [iter 9 / 42], [train loss 0.43025]
[epoch 0], [iter 12 / 42], [train loss 0.37848]
[epoch 0], [iter 15 / 42], [train loss 0.34914]
[epoch 0], [iter 18 / 42], [train loss 0.32501]
[epoch 0], [iter 21 / 42], [train loss 0.30781]
[epoch 0], [iter 24 / 42], [train loss 0.28810]
[epoch 0], [iter 27 / 42], [train loss 0.27264]
[epoch 0], [iter 30 / 42], [train loss 0.26369]
[epoch 0], [iter 33 / 42], [train loss 0.25102]
[epoch 0], [iter 36 / 42], [train loss 0.24053]
[epoch 0], [iter 39 / 42], [train loss 0.23122]
[epoch 0], [iter 42 / 42], [train loss 0.22529]


In [ ]:
total =0
correct =0
for i, data in enumerate(test_loader):
  inps, labs = data
  inps, labs = inps.cuda('cuda'), labs.cuda('cuda')

  inps.squeeze_(0) # create a mini-batch as expected by the model
  labs.squeeze_(0)
    
  # Casting to cuda variables.
  inps = Variable(inps).float().cuda('cuda')
  labs = Variable(labs).float().cuda('cuda')
  net = alexNet_model_trained.cuda('cuda')
  outs = net(inps)
  soft_outs = F.softmax(outs, dim=1)
  prds = soft_outs.data.max(1)[1]
  labs = labs.data.max(1)[1]
  for i in range(len(prds)):
    total += 1
    if prds[i] == labs[i]:
      correct += 1

acc = correct/total*100
print(f"The accuracy % is :{acc}")

The accuracy % is :92.95774647887323


## ResNet 101

In [ ]:
import torch
#model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet152', pretrained=True)
model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
print(model.fc)
model.fc = torch.nn.Linear(in_features=2048, out_features=4, bias=True)
print(model.fc)

Linear(in_features=2048, out_features=1000, bias=True)
Linear(in_features=2048, out_features=4, bias=True)


In [ ]:
# Define optimzer and criterian
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()

In [ ]:
def train(train_loader, net, criterion, optimizer, epochs, args):
  # Setting network for training mode.
  net.train()
  
  # Average Meter for batch loss.
  train_loss = list()

  for epoch in range(epochs):
    # Iterating over batches.
    for i, data in enumerate(train_loader):
      # Obtaining images, labels and paths for batch.
      inps, labs = data
      
      # Casting tensors to cuda.
      inps, labs = inps.cuda(args['device']), labs.cuda(args['device'])

      inps.squeeze_(0) # create a mini-batch as expected by the model
      labs.squeeze_(0)
        
      # Casting to cuda variables.
      inps = Variable(inps).float().cuda(args['device'])
      labs = Variable(labs).float().cuda(args['device'])

      # Clears the gradients of optimizer.
      optimizer.zero_grad()

      # # Forwarding.
      outs = net(inps)
      soft_outs = F.softmax(outs, dim=1)
      
      # Obtaining predictions.
      prds = soft_outs.data.max(1)[1]
      
      # Computing loss.
      loss = criterion(outs, labs)
              
      # Computing backpropagation.
      loss.backward()
      optimizer.step()
          
      # Appending images for epoch loss calculation.
      
      prds = prds.cpu().numpy()        
      inps_np = inps.detach().cpu().numpy()
      labs_np = labs.detach().cpu().numpy()
      
      # Updating loss meter.
      train_loss.append(loss.data.item())

      # Printing.
      if (i + 1) % args['print_freq'] == 0:
          print('[epoch %d], [iter %d / %d], [train loss %.5f]' % (epoch, i + 1, len(train_loader), np.asarray(train_loss).mean()))
      torch.cuda.empty_cache()
  return net

In [ ]:
ResNet_model_trained = train(train_loader, model.cuda('cuda'), criterion, optimizer, 1, {'device':'cuda', 'print_freq':3})

[epoch 0], [iter 3 / 42], [train loss 0.31127]
[epoch 0], [iter 6 / 42], [train loss 0.29656]
[epoch 0], [iter 9 / 42], [train loss 0.26091]
[epoch 0], [iter 12 / 42], [train loss 0.23940]
[epoch 0], [iter 15 / 42], [train loss 0.24404]
[epoch 0], [iter 18 / 42], [train loss 0.23757]
[epoch 0], [iter 21 / 42], [train loss 0.23252]
[epoch 0], [iter 24 / 42], [train loss 0.22387]
[epoch 0], [iter 27 / 42], [train loss 0.20600]
[epoch 0], [iter 30 / 42], [train loss 0.20192]
[epoch 0], [iter 33 / 42], [train loss 0.19571]
[epoch 0], [iter 36 / 42], [train loss 0.19154]
[epoch 0], [iter 39 / 42], [train loss 0.19297]
[epoch 0], [iter 42 / 42], [train loss 0.18645]


In [ ]:
total =0
correct =0
for i, data in enumerate(test_loader):
  inps, labs = data
  inps, labs = inps.cuda('cuda'), labs.cuda('cuda')

  inps.squeeze_(0) # create a mini-batch as expected by the model
  labs.squeeze_(0)
    
  # Casting to cuda variables.
  inps = Variable(inps).float().cuda('cuda')
  labs = Variable(labs).float().cuda('cuda')
  net = ResNet_model_trained.cuda('cuda')
  outs = net(inps)
  soft_outs = F.softmax(outs, dim=1)
  prds = soft_outs.data.max(1)[1]
  labs = labs.data.max(1)[1]
  for i in range(len(prds)):
    total += 1
    if prds[i] == labs[i]:
      correct += 1

acc = correct/total*100
print(f"The accuracy % is :{acc}")

The accuracy % is :84.50704225352112
